Image folder link: https://www.kaggle.com/datasets/sourcerer/flowers-recognition-4267-train-50-test-split

In [2]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
images_path = "/content/drive/My Drive/Colab Notebooks/flowers.zip"

In [7]:
from zipfile import ZipFile
with ZipFile(images_path, 'r') as zip:
  zip.extractall()

In [8]:
!ls

drive  flowers	__MACOSX  sample_data


**Importing Keras Modules**

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# **Building the CNN Classifier**

In [10]:
# Initialising the CNN classifier
classifier = Sequential()

# Add a Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu', padding = 'same'))

# Add a Max Pooling layer of size 2X2
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Add another Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))

# Adding another pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Add another Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))

# Adding another pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening the layer before fully connected layers
classifier.add(Flatten())

# Adding a fully connected layer with 512 neurons
classifier.add(Dense(units = 512, activation = 'relu'))

# Adding dropout with probability 0.5
classifier.add(Dropout(0.5))


# Adding a fully connected layer with 128 neurons
classifier.add(Dense(units = 128, activation = 'relu'))


# The final output layer with 5 neuron to predict the categorical classifcation
classifier.add(Dense(units = 5, activation = 'softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Compiling the CNN classifier with Adam optimizer (default Learning rate and other parameters) and Categorical Crossentropy as loss function and Accuracy as the metric to monitor**

Adam Configuration Parameters
1. alpha. Also referred to as the learning rate or step size. The proportion that weights are updated (e.g. 0.001). Larger values (e.g. 0.3) results in faster initial learning before the rate is updated. Smaller values (e.g. 1.0-5) slow learning right down during training
2. beta1. The exponential decay rate for the first moment estimates (e.g. 0.9).
3. beta2. The exponential decay rate for the second-moment estimates (e.g. 0.999). This value should be set close to 1.0 on problems with a sparse gradient (e.g. NLP and computer vision problems).
4. epsilon. Is a very small number to prevent any division by zero in the implementation (e.g. 10E-8).

In [13]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


# **Dataset Pre-processing**
ImageDataGenerator is a powerful preprocessing utility to generate training and testing data with common data augmentation techniques. It can also be used to generate training data from Images stored in hierarchical directory structures. For more options of ImageDataGenerator go to https://keras.io/preprocessing/image/

In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Create data generator for training data with data augmentation and normalizing all
# values by 255
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1./255)

# Setting training data generator's source directory
# Setting the target size to resize all the images to (64,64) as the model input layer expects 32X32 images

training_set = train_datagen.flow_from_directory('./flowers/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

# Setting testing data generator's source directory
test_set = test_datagen.flow_from_directory('./flowers/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')


# There are 3823 training images and 500 test images in total
classifier.fit_generator(training_set,
                         steps_per_epoch = int(3823/32),
                         epochs = 20,
                         validation_data = test_set,
                         validation_steps = int(500/32))


Found 4267 images belonging to 5 classes.
Found 50 images belonging to 5 classes.


AttributeError: 'Sequential' object has no attribute 'fit_generator'

**save the model and its weights after training**

In [37]:
classifier.save('./classifier.h5')

classifier.save_weights('./classifier_weights.weights.h5')

In [38]:
!ls

classifier.h5  classifier_weights.weights.h5  drive  flowers  __MACOSX	sample_data


# **TESTING THE MODEL**

**Loading the pretrained model**

In [39]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the pre trained model from the HDF5 file saved previously
pretrained_model = load_model('./classifier.h5')
pretrained_model.load_weights('./classifier_weights.weights.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


**Testing the model on a test image from one of the test folders**

In [40]:
import cv2
import os

# Get the absolute path to the image
image_path = os.path.abspath('./flowers/test/daisy/5547758_eea9edfd54_n.jpg')

test_image = cv2.imread(image_path)

# Check if the image was loaded correctly
if test_image is None:
    print("Error: Image could not be loaded. Check the file path.")
else:
    # ... rest of the code
    # Resize the image to 64X64 shape to be compatible with the model
    test_image = cv2.resize(test_image,(64,64))

    # Check if the size of the Image array is compatible with Keras model
    print(test_image.shape)

    # If not compatible expand the dimensions to match with the Keras Input
    test_image = np.expand_dims(test_image, axis = 0)
    test_image =test_image*1/255.0

    #Check the size of the Image array again
    print('After expand_dims: '+ str(test_image.shape))

    #Predict the result of the test image
    # 'classifier' was used instead of 'pretrained_model'
    result = pretrained_model.predict(test_image)

    # Check the indices Image Data Generator has allotted to each folder
    classes_dict = training_set.class_indices
    print(classes_dict)

    # Creating a list of classes in test set for showing the result as the folder name
    prediction_class = []
    for class_name,index in classes_dict.items():
      prediction_class.append(class_name)

    print(result[0])

    # Index of the class with maximum probability
    predicted_index = np.argmax(result[0])

    # Print the name of the class
    print(prediction_class[predicted_index])

(64, 64, 3)
After expand_dims: (1, 64, 64, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}
[0.20088059 0.19989228 0.23013908 0.19726212 0.17182587]
rose


**Generating a report on the test data**

In [41]:
# Re-initalizing the test data generator with shuffle=False to create the confusion matrix
test_set = test_datagen.flow_from_directory('./flowers/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode = 'categorical')

# Predict the whole generator to get predictions
Y_pred = classifier.predict(test_set, steps=int(500/32+1))

# Find out the predictions classes with maximum probability
y_pred = np.argmax(Y_pred, axis=1)

# Utilities for confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

# Printing the confusion matrix based on the actual data vs predicted data.
print(confusion_matrix(test_set.classes, y_pred))

# Printing the classification report
print(classification_report(test_set.classes, y_pred, target_names=prediction_class))

Found 50 images belonging to 5 classes.
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
[[ 0  1  9  0  0]
 [ 1  0  9  0  0]
 [ 0  2  8  0  0]
 [ 1  0  9  0  0]
 [ 0  0 10  0  0]]
              precision    recall  f1-score   support

       daisy       0.00      0.00      0.00        10
   dandelion       0.00      0.00      0.00        10
        rose       0.18      0.80      0.29        10
   sunflower       0.00      0.00      0.00        10
       tulip       0.00      0.00      0.00        10

    accuracy                           0.16        50
   macro avg       0.04      0.16      0.06        50
weighted avg       0.04      0.16      0.06        50



/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_clas